In [83]:
import matplotlib
matplotlib.use('Agg')
%matplotlib inline
import matplotlib.pyplot as plt

import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torch.optim as optim
from torchvision import datasets, transforms

from utils.sampling import mnist_iid, mnist_noniid, cifar_iid
from utils.options import args_parser
from utils.customloader import CustomDataset, DatasetSplit
from utils.separate_into_classes import separate_into_classes
#from utils.arguments import Args
from models.Update import LocalUpdate
from models.Nets import MLP, CNNMnist, CNNCifar
from models.Fed import FedAvg
from models.test import test_img
from utils.smooth_crossentropy import SmoothCrossEntropyLoss
from utils.dataloader import get_dataloader, set_seed
from utils.train_glob import train_global_model, test_model
import random
import os
os.environ["CUDA_VISIBLE_DEVICES"] = ""

import numpy as np
import copy
from sklearn.model_selection import train_test_split



import torch
class Args:
    #federated arugments
    epochs=5
    num_users=10
    local_ep=1
    local_bs=10
    bs=128
    lr=0.01
    momentum=0.5
    split='user'
    
    
    #model arguments
    model='mnist'
    kernel_num=9
    kernel_sizes='3,4,5'
    norm='batch_norm'
    num_filters=32
    max_pool=True
    
    #other arguments
    #data='mnist'
    #iid='store_true'
    num_channels=1
    num_classes=10
    #stopping_rounds=10
    verbose='store_true'
    seed=1
    #all_clients='store_true'
    #device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    device = torch.device('cpu')
    
    

args = Args()    
##############SET SEEDS FOR REPRODUCIBILITY#############
np.random.seed(args.seed)
random.seed(args.seed)
torch.manual_seed(args.seed)
# if you are suing GPU
torch.cuda.manual_seed(args.seed)
torch.cuda.manual_seed_all(args.seed)


torch.backends.cudnn.enabled = False 
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True    
##############~SET SEEDS FOR REPRODUCIBILITY#############

# Define Dataloader /  Model / Optimizer / Loss

In [84]:
global_train_loader, local_train_loader, test_loader = get_dataloader(data='mnist', 
                                                                      test_size=0.5, 
                                                                      num_workers=2,
                                                                      batch_size=128, 
                                                                      seed=args.seed,
                                                                     )

net_glob = CNNMnist(args=args).to(args.device)
net_glob.train()

optimizer = optim.SGD(net_glob.parameters(), lr=args.lr, momentum=args.momentum)
sloss = F.cross_entropy













(30000, 1, 28, 28)
torch.Size([30000])
(30000, 1, 28, 28)
torch.Size([30000])


# Train Global Model

In [85]:
pid = os.fork()

if pid == 0:
    print("child start")
    train_global_model(net_glob, args, global_train_loader, test_loader, sloss, optimizer)
    os._exit(127)

os.waitpid(pid,0)










child start
Train Epoch: 0 [0/30000 (0%)]	Loss: 2.350000
Train Epoch: 0 [6400/30000 (21%)]	Loss: 2.261862
Train Epoch: 0 [12800/30000 (43%)]	Loss: 1.957603
Train Epoch: 0 [19200/30000 (64%)]	Loss: 1.498059
Train Epoch: 0 [25600/30000 (85%)]	Loss: 1.169264

Train loss: 1.7320344229961964

Test set: Average loss: 0.00463 
Accuracy: 8597/10000 (85.97%)

Train Epoch: 1 [0/30000 (0%)]	Loss: 1.041325
Train Epoch: 1 [6400/30000 (21%)]	Loss: 0.691319
Train Epoch: 1 [12800/30000 (43%)]	Loss: 0.831090
Train Epoch: 1 [19200/30000 (64%)]	Loss: 0.682115
Train Epoch: 1 [25600/30000 (85%)]	Loss: 0.533530

Train loss: 0.7805149203919349

Test set: Average loss: 0.00258 
Accuracy: 9085/10000 (90.85%)

Train Epoch: 2 [0/30000 (0%)]	Loss: 0.567517
Train Epoch: 2 [6400/30000 (21%)]	Loss: 0.590767
Train Epoch: 2 [12800/30000 (43%)]	Loss: 0.570652
Train Epoch: 2 [19200/30000 (64%)]	Loss: 0.533895
Train Epoch: 2 [25600/30000 (85%)]	Loss: 0.746364

Train loss: 0.5727267388333666

Test set: Average loss: 0.001

(12820, 32512)

# Test model after training on global data

In [86]:

#Before fedlearning
path_checkpoint='./test'

net_glob.load_state_dict(torch.load(path_checkpoint))
print('Before Federated Learning')
test_model(net_glob, test_loader, sloss)


Before Federated Learning

Test set: Average loss: 0.00127 
Accuracy: 9500/10000 (95.00%)



In [87]:
checkpoint_globalnet = copy.deepcopy(net_glob)

# Define Distribution of Data

In [88]:
dict_users = mnist_iid(global_train_loader.dataset.data, args.num_users)
len(dict_users[1])

3000

In [89]:
unique, counts = np.unique(global_train_loader.dataset.targets, return_counts=True)
print(unique)
print(counts)
sorted_y = copy.deepcopy(global_train_loader.dataset.targets)
sorted_index_y = np.argsort(np.squeeze(sorted_y))

class_dist=[]

for i in range(args.num_classes):
    print(i)
    class_dist.append(np.array(sorted_index_y[sum(counts[:i]):sum(counts[:i+1])], dtype=np.int64))
    
non_iid = np.array(class_dist)

[0 1 2 3 4 5 6 7 8 9]
[2961 3371 2979 3066 2921 2711 2959 3132 2926 2974]
0
1
2
3
4
5
6
7
8
9


In [90]:
individual = []
for j in range(10):
    individual.append(np.array_split(class_dist[j], 10))

user_dist=[]
for i in range(10):
    temp=[]
    for j in range(10):
        temp.append(individual[j][i])
        
    
    user_dist.append((np.concatenate(temp)).astype(np.int64))    
    
iid=np.array(user_dist)

# Train Local Model

In [91]:
from utils.multiprocessing import work, multi_train_local_dif
from torch.utils.data.sampler import Sampler
from torchvision import datasets, transforms
import torch.multiprocessing as mp

In [92]:
if __name__ == '__main__':
    
    mp.set_start_method('fork', force=True)
    sloss2 = F.cross_entropy
    torch.set_num_threads(1)
    
    checkpoint_globalnet11 = copy.deepcopy(net_glob)
    
    for i in range(args.local_ep + 3):
        
        print('--------------------------------------------')
        print("\n\n\nstart training epoch : " + str(i) + "\n\n\n")
        print('--------------------------------------------')
        
        procs=[]
        loss_locals=[]
        w_locals=[]        
        
        q_l = mp.Queue()
        q_w = mp.Queue()        
        
        for i in range(args.num_users):

            p = mp.Process(target=multi_train_local_dif, args=(q_l, q_w, args, i, sloss2, local_train_loader, non_iid, checkpoint_globalnet11))
            procs.append(p)
            p.start()

        for p in procs:
            loss_locals.append(q_l.get(p))
            w_locals.append(q_w.get(p))

        for p in procs:
            p.join()

            
        print('--------------------------------------------\n\n')
        w_glob = FedAvg(w_locals)
        checkpoint_globalnet11.load_state_dict(w_glob)
        test_model(checkpoint_globalnet11, test_loader, sloss)
        print('\n\n--------------------------------------------')
        





--------------------------------------------



start training epoch : 0



--------------------------------------------
training user: 0
training user: 1
training user: 2
training user: 3
training user: 4
training user: 5
training user: 6
training user: 7
training user: 8
training user: 9
Local user 0 -- Update Epoch: 0 [0/2961 (0%)]	Loss: 0.047337
Local user 1 -- Update Epoch: 0 [0/3371 (0%)]	Loss: 0.263768
Local user 0 -- Update Epoch: 0 [100/2961 (3%)]	Loss: 0.023780
Local user 2 -- Update Epoch: 0 [0/2979 (0%)]	Loss: 0.163452
Local user 3 -- Update Epoch: 0 [0/3066 (0%)]	Loss: 0.594407
Local user 4 -- Update Epoch: 0 [0/2921 (0%)]	Loss: 0.192766
Local user 5 -- Update Epoch: 0 [0/2711 (0%)]	Loss: 0.619774
Local user 6 -- Update Epoch: 0 [0/2959 (0%)]	Loss: 0.248543
Local user 7 -- Update Epoch: 0 [0/3132 (0%)]	Loss: 0.173744
Local user 8 -- Update Epoch: 0 [0/2926 (0%)]	Loss: 0.199489
Local user 9 -- Update Epoch: 0 [0/2974 (0%)]	Loss: 0.716594
Local user 1 -- Update Epoch: 0 [100

# Test Model on local data

In [95]:
#After fedlearning
print('Before Federated Learning')
test_model(net_glob, test_loader, sloss)

Before Federated Learning

Test set: Average loss: 0.00127 
Accuracy: 9500/10000 (95.00%)



In [96]:
#After fedlearning
print('After Federated Learning -- checkpoint')
test_model(checkpoint_globalnet11, test_loader, sloss)

After Federated Learning -- checkpoint

Test set: Average loss: 0.00154 
Accuracy: 9345/10000 (93.45%)

